alignment in time and space: done after correction and filtering 
;before calculating other parameters

In [1]:
import numpy as np
import os
import pandas as pd
from trajalign.traj import Traj
from trajalign.average import load_directory


In [2]:
# get all traj in a list and align => see pretreatment BUT with only 0,1,2,3 as columns 
#files containing fusion events folder
path='/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/' 

files = ['Prm1'] 

#color of h-/h+ spatial marker
RefM= '_B' #'_B' if POI in h+ cell, '_R' if POI in h- cell
RefP= '_R' #'_R' if POI in h+ cell, '_B' if POI in h- cell
    
def replicats_nb(POI):
    rep_list = [ folder for folder in os.listdir(path+POI) if 'Rep_' in folder ]
    r= len(rep_list)
    return r

def get_data(POI,rep) :
    directory_rep = os.listdir( path+POI+'/Rep_'+str(rep+1)+'/trajectories') #absolute path of directory to charge data from
    f = [ folder for folder in directory_rep if 'cell_' in folder ] # 'cell_' -> folder name to list
    return f 

#determin fusion frame based on cytoplasmic fluorescence intensity
def fusion_frame (m_c):
    for x in m_c:
        n=len(x)   # number of frames
        U=np.empty( ((n-2),2) ) # creation of an empty array of n-2 lines and 2 columns
        
            
        for tau in range ((n-2)):  
# construction of the model for frame = x            
            m1 = np.mean(x.f()[0:tau])   
            m2 = np.mean(x.f()[(tau+1):n])
            m = np.array([m1] * tau + [m2]* (n-tau)) # use np.array because python can't substract list
            e = x.f() - m                           # substract theorical model from data
            U[tau,0] = x.frames()[tau]               # first column of matrix = frame number
            U[tau,1] = np.sum(e**2)                  # second column of matrix = sum of square errors 
        pos= np.argmin( U[:,1])              # get position of frame corresponding to mimimum sum of squared errors
        f_f=x.frames()[pos]              #get frame of fusion
        f_f= f_f-1   #correct for diff frame numeration PT/Fiji
        
    return f_f 

#align trajectory such that centermass trajectory from h- cell = origin 
def fusion_axis( t_l ) :
	# input a trajectory list t_l

	# search for the red trajectory, and compute its
	# center of mass
    for t in t_l :
	
		# select the blue trajectory
        if RefM in t.annotations()[ 'file' ] :
			# compute the center of mass of the trajctory
            r = t.center_mass()

	# subtract the center of mass of the red trajectory
	# from all trajectories in the cell. The center of mass
	# of the red trajectory will definethe origin of the x-y
	# plane
    for t in t_l :
		
		# translate all the trajectories accordingly
        t.translate( -r )
		# normalise the flurescence intensities between 0 and 1
        t.norm_f() # normalise between 0 and 1

	# compute the center of mass of the blue trajectory.
	# As the trajectory has been now translated by r, 
	# its center of mass defines the vector of the cell
	# fusion reference frame defined by the two center of
	# mass positions of the Myo52-RFP and Myo52-BFP
    for t in t_l :

		# select the red trajectory
        if RefP in t.annotations()[ 'file' ] :
			# compute the center of mass of the trajctory
            b = t.center_mass()
			# normalise the vector to a unit vector
            v = [ i / np.sqrt( b[ 0 ] ** 2 + b[ 1 ] ** 2 ) for i in b ]

    return t_l , v 
           

In [3]:
n= len(files) 
Rep= []
for i in range(n):
    r=replicats_nb(files[i])
    Rep.append(r)
folders=[[] for _ in range(n)] #lists for POI rep folders => each subset will contains n subset for each replicat

for i in range(len(Rep)):
    folders[i]=[[] for _ in range(Rep[i])] 

# list all cell_ folders for each POI 

for i in range(n) :
    for r in range(Rep[i]):
        f=get_data(files[i],r)
        folders[i][r]= f 
        
# get trajectories for each fusions sites of each POI
# trajectory classes => frames, coord = centroid coord; coord_err= [0]mu20 et [1] mu02; f_err= mu11 

ct=0
for f in folders:
    r=1
    file_path=os.path.join(path,files[ct])
    for rep in f:
        id=0
        replicat='Rep_'+str(r)
        event_path=os.path.join(file_path,replicat)+'/trajectories/'
        print(event_path)

        for event in rep: 
            t_br = load_directory(os.path.join(event_path,event), pattern= 'Traj_' , comment_char = '#' , frames = 0 , coord = ( 1 , 2 ) , f = 3 ,coord_unit = 'pxl' )
            for tr in t_br:
                if RefM in tr.annotations()[ 'file' ] :
                    tr.save(os.path.join(event_path,event,'Trajectory_B.txt'))
                elif RefP in tr.annotations()[ 'file' ] :
                    tr.save(os.path.join(event_path,event,'Trajectory_R.txt'))
                    
                    
            m_c = load_directory(os.path.join(event_path,event), pattern= '.csv',  comment_char = '#' ,sep= ',', frames = 0 , f = 1 )
            t_l = load_directory(os.path.join(event_path,event), pattern= 'Trajectory' , comment_char = '#' , frames = 0 , coord = ( 1 , 2 ) , f = 3 ,coord_unit = 'pxl' )   
            f_f = fusion_frame (m_c)
            for c in m_c:
                centered_c= c.frames()-f_f
                c.input_values('frames',centered_c)
                c.time (3, min) 
                c.norm_f() 
        #center trajectory in time 
            for t in t_l:
                centered_f= t.frames()-f_f     #remove f_f from frame number to center around fusion time (=0)
                t.input_values('frames',centered_f) #replace fram number in trajectories by centered frame number
                scaled_t= t.coord()/ (15.4321)
                t.input_values('coord', scaled_t)   # scale: 15.4321 px/µm, get coordonated as µm
                t.time (3, min)                       # enter time value for each point of the trajectory
            t_l_0 , v = fusion_axis( t_l )

	# rotate the trajectory, so that the fusion axis is 
	# aligned with x positive axis
            angle = np.sign( v[ 0 ] ) * np.arcsin( v[ 1 ] ) - ( np.sign( v[ 0 ] ) - 1 ) * np.pi / 2 
	# add angle offset (we want to see red on the left of the plot, blue on the right)
            angle = angle + np.pi 

	# rotate each trajectory in the trajectory list + save it in new folder 
            os.makedirs(path+files[ct]+'/aligned_trajectories/Rep_'+str(r)+'/cell_%03d/' %id, exist_ok=True)
            print(path+files[ct]+'/aligned_trajectories/Rep_'+str(r)+'/cell_%03d/' %id)
            for t_0 in t_l_0 :
                t_0.rotate( -angle )
                if RefM in t_0.annotations()[ 'file' ] :
                    t_0.save(file_path+'/aligned_trajectories/Rep_'+str(r)+'/cell_%03d/Traj_M.txt'%id)
                elif RefP in t_0.annotations()[ 'file' ] :
                    t_0.save(file_path+'/aligned_trajectories/Rep_'+str(r)+'/cell_%03d/Traj_P.txt'%id)
                else :
                    t_0.save(file_path+'/aligned_trajectories/Rep_'+str(r)+'/cell_%03d/Traj_POI.txt'%id)
                    print(file_path+'/aligned_trajectories/Rep_'+str(r)+'/cell_%03d/Traj_POI.txt'%id)
        
            id+=1
        r+=1
    ct+=1

/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1/Rep_1/trajectories/
1-Traj_01_B-01.txt
1-Traj_01_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-mCherry_01.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Trajectory_B.txt
Trajectory_G.txt
Trajectory_R.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1/aligned_trajectories/Rep_1/cell_000/
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1/aligned_trajectories/Rep_1/cell_000/Traj_POI.txt
1-Traj_03_B-01.txt
1-Traj_03_R-03.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-mCherry_03.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Trajectory_B.txt
Trajectory_G.txt
Trajectory_R.txt

 >> load_directory: T

/mnt/c/Users/Valentine/Documents/Scripts/notebook/jupyterenv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/c/Users/Valentine/Documents/Scripts/notebook/jupyterenv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-mCherry_15.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Trajectory_B.txt
Trajectory_G.txt
Trajectory_R.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1/aligned_trajectories/Rep_1/cell_011/
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1/aligned_trajectories/Rep_1/cell_011/Traj_POI.txt
1-Traj_18_B-02.txt
1-Traj_18_R-03.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-mCherry_18.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Trajectory_B.txt
Trajectory_G.txt
Trajectory_R.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_spread

/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1/aligned_trajectories/Rep_1/cell_037/
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1/aligned_trajectories/Rep_1/cell_037/Traj_POI.txt
3-Traj_01_B-01.txt
3-Traj_01_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

3-mCherry_01.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Trajectory_B.txt
Trajectory_G.txt
Trajectory_R.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1/aligned_trajectories/Rep_1/cell_038/
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1/aligned_trajectories/Rep_1/cell_038/Traj_POI.txt
3-Traj_03_B-01.txt
3-Traj_03_R-02.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

3-mCherry_03.csv

 >> load_directory: The 'intensity

/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1/aligned_trajectories/Rep_1/cell_067/Traj_POI.txt
4-Traj_12_B-01.txt
4-Traj_12_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

4-mCherry_12.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Trajectory_B.txt
Trajectory_G.txt
Trajectory_R.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1/aligned_trajectories/Rep_1/cell_068/
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1/aligned_trajectories/Rep_1/cell_068/Traj_POI.txt
4-Traj_13_B-02.txt
4-Traj_13_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

4-mCherry_13.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Trajectory_B.txt
Trajectory_G.txt
Trajectory